<a href="https://colab.research.google.com/github/JianfengMI/MLprojects/blob/main/finance/stock_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try to use Sentiment analysis to predict stock price
- retrieve stock price
- sentiment analysis - retrieve text and converting it into a numeric value to represent the feeling of the text

## install / import packages

In [ ]:
! pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
## Example pulling microsoft stock
msft = yf.Ticker("MSFT")

In [ ]:
hist = msft.history(period="max")

In [ ]:
hist.head()

In [ ]:
hist.Open.plot(figsize=(12,4), title="Microsoft Stock Price")
plt.show()

In [ ]:
# pull more stocks (3 years of history)
stocks = [
    "^GSPC",
    "ETSY",
    "PINS",
    "TSLA",
    "SHOP",
    "O",
    "MELI",
    "ISRG",
    "DIS",
    "BRK-B",
    "AMZN",
    "ZM",
    "PFE",
    "CLX",
    "DPZ",
    "RTX",
]

In [ ]:
hists = {}
for stock in stocks:
    hists[stock] = yf.Ticker(stock).history(period="3y")

In [ ]:
# plot the stock price with plotly
for stock in stocks:
  temp_df = hists[stock].copy()

  fig = go.Figure(
      data=[go.Candlestick(
          x=temp_df.index,
          open=temp_df["Open"],
          high=temp_df["High"],
          low=temp_df["Low"],
          close=temp_df["Close"]
      )]
  )
  fig.update_layout(title=stock, xaxis_rangeslider_visible=False)
  fig.show()

## pull Tweets about each stock

In [ ]:
# try CLX fisrt
hists["CLX"].index.min()

In [ ]:
stock = "CLX"

In [ ]:
! pip install snscrape

In [ ]:
# import libraries
import tweepy
import os
import pandas as pd
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter

# scrape data from tweeter
tweets_list = []

for i, tweet in tqdm(
    enumerate(
        sntwitter.TwitterSearchScraper(
            f"${stock} since:2022-06-27 until:2025-06-24"
        ).get_items()),
    total=1000):
    if i > 1000:
      break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username])


tweets_df = pd.DataFrame(tweets_list, columns=["date", "id", "content", "username"])

In [ ]:
## since snscrape doesn't work anymore, try using 'tweepy' library. Note that you’ll need X API credentials (API Key, API Secret, Access Token, and Access Token Secret) to use this script.

In [ ]:
import tweepy
import pandas as pd
from datetime import datetime
import os

# Replace these with your own X API credentials
API_KEY = "7xcxm004fsnWToVBd4ej5D5cz"
API_SECRET = "wh1wLvROvzQ9iGw0TGbutiaBIiNH09jLsrhkGJGc1tRc5JB1j3"
ACCESS_TOKEN = "254562826-9YQT8qvsQdSn8Bb4OXZhN17Rymg06gvb7qtg2DGW"
ACCESS_TOKEN_SECRET = "XEDutvVqTxWnzJeWiIIRBf2r444MpqlHzsEUJVqrjFHHn"

# Authenticate with X API
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Define the search query and date range
search_query = "$CLX -filter:retweets"  # Search for $CLX, exclude retweets
since_date = "2022-06-25"
until_date = "2025-06-25"  # Current date as of request

# Function to convert date to X API format (YYYYMMDDHHMM)
def format_date_for_api(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    return date_obj.strftime("%Y%m%d%H%M")

# Initialize a list to store tweet data
tweets_data = []

# Search for tweets
try:
    for tweet in tweepy.Cursor(api.search_tweets,
                             q=search_query,
                             since_id=None,
                             until=format_date_for_api(until_date),
                             lang="en").items(1000):  # Limit to 1000 tweets for this example
        tweet_date = tweet.created_at
        if tweet_date >= datetime.strptime(since_date, "%Y-%m-%d"):
            tweets_data.append({
                "created_at": tweet.created_at,
                "username": tweet.user.screen_name,
                "text": tweet.text,
                "retweet_count": tweet.retweet_count,
                "favorite_count": tweet.favorite_count
            })

    # Create a DataFrame
    df = pd.DataFrame(tweets_data)

    # Save to CSV
    output_file = "clx_threads_2022_06_25_to_2025_06_25.csv"
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

except tweepy.TweepyException as e:
    print(f"Error occurred: {e}")

# it looks X free account doens't have access to retrieving posts. Have to change to other approaches
-let's try using yahoo finance headlines

In [ ]:
# install necessary packages
!pip install transformers feedparser requests

In [ ]:
import feedparser
import requests
from transformers import pipeline

In [ ]:
# use <finbert> model
pipe = pipeline("text-classification", model="ProsusAI/finbert")

In [ ]:
print(pipe("stocks rallied and the british pound gained."))

In [ ]:
ticker = "META"
keyword = 'meta'

rss_url = f"https://finance.yahoo.com/rss/headline?s={ticker}"
feed = feedparser.parse(rss_url)

total_score = 0
num_articles = 0

for i, entry in enumerate(feed.entries):
    if keyword.lower() not in entry.title.lower():
        continue
    else:
        print(f'Title: {entry.title}')
        print(f'Link: {entry.link}')
        print(f'Published: {entry.published}')
        print(f'Summary: {entry.summary}')

        sentiment = pipe(entry.summary)[0]

        print(f'Sentiment: {sentiment["label"]}, Score: {sentiment["score"]}')
        print('-' * 50)

        if sentiment['label'] == 'positive':
            total_score += sentiment['score']
            num_articles += 1
        elif sentiment['label'] == 'negative':
            total_score -= sentiment['score']
            num_articles += 1
final_score = total_score / num_articles if num_articles > 0 else 0
print(f'Overall sentiment:{"Positive" if final_score >= 0.15 else "Negative" if final_score <=-0.15 else "Neutral"} {final_score}')

In [ ]:
keyword = 'tesla'
date = '2025-06-24' # free version only allow to access ONE month old
API_KEY = '71e85c59c5ec44a798757332a7a20ca2'

url = (
    'https://newsapi.org/v2/everything?'
    f'q={keyword}&'
    f'from={date}&'
    'sortBy=popularity&'
    f'apiKey={API_KEY}'
)
response = requests.get(url)
articles = response.json()["articles"]
articles = [article for article in articles if keyword.lower() in article["title"].lower() or keyword.lower() in article["description"].lower()]

total_score = 0
num_articles = 0

for i, article in enumerate(articles):
        print(f'Title: {article["title"]}')
        print(f'Link: {article["url"]}')
        print(f'Description: {article["description"]}')

        sentiment = pipe(article['content'])[0]

        print(f'Sentiment: {sentiment["label"]}, Score: {sentiment["score"]}')
        print('-' * 50)

        if sentiment['label'] == 'positive':
            total_score += sentiment['score']
            num_articles += 1
        elif sentiment['label'] == 'negative':
            total_score -= sentiment['score']
            num_articles += 1
final_score = total_score / num_articles if num_articles > 0 else 0
print(f'Overall sentiment:{"Positive" if final_score >= 0.15 else "Negative" if final_score <=-0.15 else "Neutral"} {final_score}')